# Baseline Evaluation: Gemini vs Base Qwen

This notebook evaluates the baseline performance of:
1. **Gemini 2.5 Flash-Lite** (current 2-step pipeline)
2. **Qwen3-4B-Instruct-2507** (base model, untuned)

Metrics tracked:
- Syntactic validity (passes RuleBasedValidator)
- Execution success (runs on Neo4j)
- Semantic accuracy (exact result match with gold Cypher)
- Latency (ms)
- Token usage (input/output tokens)


In [8]:
# Setup & Configuration

import json
import os
import sys
import time
from collections import defaultdict
from pathlib import Path
from typing import Any

import pandas as pd
import tiktoken
from dotenv import load_dotenv
from tqdm import tqdm

# Add project root to path
# In Jupyter notebooks, we need to walk up until we find the project root
# The project root should have both pyproject.toml AND src/ directory
import os

# Manual override: If you know the project root path, uncomment and set it directly
# ROOT = Path(r"C:\Users\ishui\Desktop\OncoGraph\OncoGraph Agent").resolve()

# Auto-detect by walking up from current directory
# Start from current working directory (usually where notebook is located)
ROOT = Path(os.getcwd()).resolve()

# Walk up until we find a directory with both pyproject.toml AND src/
original_cwd = ROOT
found_root = False
while True:
    # Check if this directory looks like the project root
    has_pyproject = (ROOT / "pyproject.toml").exists()
    has_src = (ROOT / "src").exists() and (ROOT / "src").is_dir()
    
    if has_pyproject and has_src:
        found_root = True
        break
    
    # Go up one level
    parent = ROOT.parent
    if parent == ROOT:  # Reached filesystem root
        break
    ROOT = parent

if not found_root:
    raise FileNotFoundError(
        f"Could not find project root (directory with both pyproject.toml and src/). "
        f"Current directory: {os.getcwd()}. "
        f"Please uncomment and set ROOT manually above."
    )

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print(f"Project root: {ROOT}")
print(f"Current working directory: {os.getcwd()}")
print(f"src module verified at: {ROOT / 'src'}")

from src.pipeline.executor import Neo4jExecutor
from src.pipeline.gemini import (
    GeminiConfig,
    GeminiCypherGenerator,
    GeminiInstructionExpander,
)
from src.pipeline.types import PipelineConfig, PipelineError
from src.pipeline.validator import RuleBasedValidator

load_dotenv()

# Configuration
TEST_SUBSET_SIZE = 200  # Number of test records to evaluate
RATE_LIMIT_RPM = 15  # Gemini rate limit: requests per minute
RATE_LIMIT_RPD = 1000  # Gemini rate limit: requests per day

# Path configuration (adjust for Colab/local)
# For Colab: uncomment and adjust Google Drive paths
# from google.colab import drive
# drive.mount('/content/drive')
# BASE_DIR = Path("/content/drive/MyDrive/OncoGraph-Agent")

# For local: use project root
BASE_DIR = ROOT

DATA_DIR = BASE_DIR / "finetuning" / "dataset" / "splits"
EVAL_DIR = BASE_DIR / "finetuning" / "eval"
EVAL_DIR.mkdir(parents=True, exist_ok=True)

# Environment variables
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.environ.get("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD", "")
GEMINI_API_KEY = os.environ.get("GOOGLE_API_KEY", "")

if not NEO4J_PASSWORD:
    raise ValueError("NEO4J_PASSWORD environment variable not set")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY environment variable not set")

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Evaluation directory: {EVAL_DIR}")
print(f"Test subset size: {TEST_SUBSET_SIZE}")


Project root: C:\Users\ishui\Desktop\OncoGraph\OncoGraph Agent
Current working directory: c:\Users\ishui\Desktop\OncoGraph\OncoGraph Agent\finetuning\notebooks
src module verified at: C:\Users\ishui\Desktop\OncoGraph\OncoGraph Agent\src
Base directory: C:\Users\ishui\Desktop\OncoGraph\OncoGraph Agent
Data directory: C:\Users\ishui\Desktop\OncoGraph\OncoGraph Agent\finetuning\dataset\splits
Evaluation directory: C:\Users\ishui\Desktop\OncoGraph\OncoGraph Agent\finetuning\eval
Test subset size: 200


In [9]:
# Helper Functions

from collections import Counter


def load_test_set(n: int) -> list[dict[str, Any]]:
    """Load test set and optionally sample n records with stratified sampling."""
    test_file = DATA_DIR / "test_sample.jsonl"
    if not test_file.exists():
        raise FileNotFoundError(f"Test file not found: {test_file}")
    
    records = []
    with test_file.open("r", encoding="utf-8") as f:
        for line in f:
            records.append(json.loads(line))
    
    if n >= len(records):
        return records
    
    # Stratified sampling by template_id
    records_by_template = defaultdict(list)
    for record in records:
        records_by_template[record["template_id"]].append(record)
    
    sampled = []
    for _template_id, template_records in records_by_template.items():
        num_to_sample = max(1, int(len(template_records) * (n / len(records))))
        if num_to_sample > len(template_records):
            num_to_sample = len(template_records)
        sampled.extend(template_records[:num_to_sample])
    
    # If we still need more, randomly sample remaining
    if len(sampled) < n:
        remaining = [r for r in records if r not in sampled]
        import random
        random.seed(42)
        additional = random.sample(remaining, n - len(sampled))
        sampled.extend(additional)
    
    return sampled[:n]


def evaluate_cypher_syntax(
    cypher: str, validator: RuleBasedValidator
) -> tuple[bool, str | None]:
    """Check if Cypher passes syntactic validation."""
    try:
        validator.validate_cypher(cypher)
        return True, None
    except PipelineError as e:
        return False, str(e)
    except Exception as e:
        return False, f"Unexpected error: {type(e).__name__}: {e}"


def evaluate_cypher_execution(
    cypher: str, executor: Neo4jExecutor
) -> tuple[bool, list[dict[str, Any]], str | None]:
    """Execute Cypher and return results."""
    try:
        rows = executor.execute_read(cypher)
        return True, rows, None
    except Exception as e:
        return False, [], f"{type(e).__name__}: {e}"


def normalize_result_row(row: dict[str, Any]) -> dict[str, Any]:
    """Normalize arrays and None values for comparison."""
    normalized = {}
    for key, value in row.items():
        if isinstance(value, list):
            normalized[key] = sorted(value) if value else []
        elif value is None:
            normalized[key] = None
        else:
            normalized[key] = value
    return normalized


def compare_results(gold_rows: list[dict], generated_rows: list[dict]) -> bool:
    """Compare result sets for exact match (order-independent)."""
    if len(gold_rows) != len(generated_rows):
        return False
    
    # Normalize and sort rows for comparison
    gold_normalized = sorted(
        [tuple(sorted(normalize_result_row(row).items())) for row in gold_rows]
    )
    gen_normalized = sorted(
        [tuple(sorted(normalize_result_row(row).items())) for row in generated_rows]
    )
    
    return gold_normalized == gen_normalized


def count_tokens(text: str, encoding_name: str = "o200k_base") -> int:
    """Count tokens using tiktoken."""
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(text))


# Initialize tiktoken encoders
GEMINI_ENCODER = tiktoken.get_encoding("o200k_base")  # Compatible with Gemini
# For Qwen, we'll use the tokenizer directly since it's available via transformers

print("Helper functions loaded successfully")


Helper functions loaded successfully


In [10]:
# Evaluation Harness

class Evaluator:
    """Evaluation harness for comparing generated Cypher against gold standard."""
    
    def __init__(
        self,
        validator: RuleBasedValidator,
        executor: Neo4jExecutor,
        token_encoder: tiktoken.Encoding,
    ):
        self.validator = validator
        self.executor = executor
        self.token_encoder = token_encoder
    
    def evaluate_single(
        self,
        question: str,
        gold_cypher: str,
        generated_cypher: str,
        prompt_text: str,
        model_name: str,
    ) -> dict[str, Any]:
        """Evaluate a single generated Cypher query."""
        start_time = time.perf_counter()
        
        # Count tokens
        input_tokens = len(self.token_encoder.encode(prompt_text))
        output_tokens = len(self.token_encoder.encode(generated_cypher))
        
        # Syntactic validation
        syntactic_valid, syntax_error = evaluate_cypher_syntax(
            generated_cypher, self.validator
        )
        
        # Execution (only if syntactically valid)
        execution_success = False
        execution_error = None
        generated_rows = []
        if syntactic_valid:
            execution_success, generated_rows, execution_error = evaluate_cypher_execution(
                generated_cypher, self.executor
            )
        
        # Get gold results (execute once and cache)
        _, gold_rows, _ = evaluate_cypher_execution(gold_cypher, self.executor)
        
        # Result comparison (only if both executed successfully)
        result_match = False
        if syntactic_valid and execution_success:
            result_match = compare_results(gold_rows, generated_rows)
        
        latency_ms = (time.perf_counter() - start_time) * 1000
        
        return {
            "syntactic_valid": syntactic_valid,
            "execution_success": execution_success,
            "result_match": result_match,
            "generated_cypher": generated_cypher,
            "error": syntax_error or execution_error,
            "latency_ms": latency_ms,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "gold_rows": gold_rows,
            "generated_rows": generated_rows,
        }
    
    def aggregate_metrics(self, results: list[dict[str, Any]]) -> dict[str, Any]:
        """Aggregate metrics from evaluation results."""
        total = len(results)
        if total == 0:
            return {}
        
        syntactic_valid_count = sum(1 for r in results if r["syntactic_valid"])
        execution_success_count = sum(1 for r in results if r["execution_success"])
        result_match_count = sum(1 for r in results if r["result_match"])
        
        total_input_tokens = sum(r["input_tokens"] for r in results)
        total_output_tokens = sum(r["output_tokens"] for r in results)
        total_latency_ms = sum(r["latency_ms"] for r in results)
        
        return {
            "total": total,
            "syntactic_validity_pct": (syntactic_valid_count / total) * 100,
            "execution_success_pct": (
                (execution_success_count / syntactic_valid_count) * 100
                if syntactic_valid_count > 0
                else 0.0
            ),
            "semantic_accuracy_pct": (
                (result_match_count / execution_success_count) * 100
                if execution_success_count > 0
                else 0.0
            ),
            "avg_latency_ms": total_latency_ms / total,
            "avg_input_tokens": total_input_tokens / total,
            "avg_output_tokens": total_output_tokens / total,
            "total_input_tokens": total_input_tokens,
            "total_output_tokens": total_output_tokens,
        }

print("Evaluation harness ready")


Evaluation harness ready


In [11]:
# Load Test Set

test_records = load_test_set(TEST_SUBSET_SIZE)
print(f"Loaded {len(test_records)} test records")
print("Template distribution:")
template_counts = Counter(r["template_id"] for r in test_records)
for template_id, count in sorted(template_counts.items()):
    print(f"  {template_id}: {count}")


Loaded 200 test records
Template distribution:
  f1_1_targets_gene: 6
  f1_2_genes_for_therapy: 6
  f1_3_gene_of_variant: 7
  f1_4_variants_of_gene: 7
  f2_1_targets_properties: 5
  f2_2_gene_affects_therapy_disease: 11
  f2_2_gene_affects_therapy_disease_resistance: 12
  f2_2_gene_affects_therapy_disease_sensitivity: 13
  f2_2_variant_affects_therapy_disease: 12
  f2_2_variant_token_affects_therapy_disease: 11
  f2_3_evidential_pmids_variant_resistance: 5
  f2_3_evidential_pmids_variant_sensitivity: 5
  f2_4_variant_node_property: 5
  f3_1_therapies_union_genes: 2
  f3_2_therapies_intersection_genes: 5
  f3_3_therapies_target_a_not_b: 5
  f4_1_target_validation_gene_disease: 11
  f4_2_alternative_therapies_from_resistance_genes: 11
  f5_1_biomarkers_in_disease: 11
  f5_2_therapies_with_biomarker_evidence_in_disease: 11
  f6_1_resistance_genes_for_union_therapies_in_disease: 11
  f6_2_therapies_target_gene_not_other_with_disease_evidence: 5
  f6_3_therapies_target_gene_with_variant_res

In [12]:
# Initialize Gemini Components

config = PipelineConfig()
gemini_config = GeminiConfig(
    model="gemini-2.5-flash-lite",
    temperature=0.1,
    api_key=GEMINI_API_KEY,
)

expander = GeminiInstructionExpander(config=gemini_config)
generator = GeminiCypherGenerator(config=gemini_config)
validator = RuleBasedValidator(config=config)
executor = Neo4jExecutor(
    uri=NEO4J_URI,
    user=NEO4J_USER,
    password=NEO4J_PASSWORD,
    config=config,
)

# Create evaluator
evaluator = Evaluator(validator, executor, GEMINI_ENCODER)

print("Gemini components initialized")
print(f"Model: {gemini_config.model}")


Gemini components initialized
Model: gemini-2.5-flash-lite


In [ ]:
# Gemini Evaluation

from time import sleep
import re
import random

def extract_retry_delay(error_str: str) -> float | None:
    """Extract retry delay from Gemini API error message."""
    # Look for "Please retry in X.XXXs" pattern
    match = re.search(r"Please retry in ([\d.]+)s", error_str, re.IGNORECASE)
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            pass
    # Look for RetryInfo retryDelay in details
    match = re.search(r"'retryDelay':\s*['\"]?(\d+)s", error_str, re.IGNORECASE)
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            pass
    return None

def call_with_retry(func, *args, max_attempts=5, base_delay=1.0, **kwargs):
    """Call function with exponential backoff, respecting API RetryInfo if available."""
    last_exception = None
    for attempt in range(max_attempts):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            last_exception = e
            error_str = str(e)
            
            # Check for rate limit (429)
            if "429" in error_str or "RESOURCE_EXHAUSTED" in error_str:
                # Extract retry delay from API response
                retry_delay = extract_retry_delay(error_str)
                if retry_delay:
                    delay = retry_delay + 1.0  # Add buffer
                    print(f"\\nRate limit hit. API recommends retry in {retry_delay:.1f}s. Waiting {delay:.1f}s...")
                else:
                    # Fallback to exponential backoff with jitter
                    delay = base_delay * (2 ** attempt) + random.uniform(0, 1)
                    print(f"\\nRate limit hit (attempt {attempt + 1}/{max_attempts}). Waiting {delay:.1f}s...")
                
                if attempt < max_attempts - 1:
                    sleep(delay)
                    continue
            
            # For other errors, use exponential backoff
            if attempt < max_attempts - 1:
                delay = base_delay * (2 ** attempt)
                print(f"\\nError (attempt {attempt + 1}/{max_attempts}): {type(e).__name__}. Retrying in {delay:.1f}s...")
                sleep(delay)
                continue
            
            # Last attempt failed
            raise last_exception
    
    raise last_exception

random.seed(42)  # For jitter consistency

gemini_results = []
rate_limit_timestamps = []  # Track request timestamps for rate limiting
checkpoint_file = EVAL_DIR / "gemini_results_checkpoint.jsonl"

# Load checkpoint if exists
if checkpoint_file.exists():
    print(f"Loading checkpoint from {checkpoint_file}")
    with checkpoint_file.open("r", encoding="utf-8") as f:
        checkpoint_records = [json.loads(line) for line in f]
        gemini_results = checkpoint_records
        # Extract processed record IDs
        processed_ids = {r["id"] for r in checkpoint_records}
        # Filter out already processed records
        test_records = [r for r in test_records if r["id"] not in processed_ids]
        print(f"Resuming from checkpoint: {len(checkpoint_records)} already processed, {len(test_records)} remaining")

preview_test_records = test_records[:10]

for i, record in enumerate(tqdm(preview_test_records, desc="Evaluating Gemini")):
    question = record["question"]
    gold_cypher = record["cypher"]
    record_id = record["id"]
    
    # Rate limiting: ensure we don't exceed 15 RPM
    current_time = time.time()
    rate_limit_timestamps = [
        ts for ts in rate_limit_timestamps if current_time - ts < 60
    ]  # Keep only timestamps from last minute
    
    if len(rate_limit_timestamps) >= RATE_LIMIT_RPM:
        sleep_time = 60 - (current_time - rate_limit_timestamps[0]) + 1
        if sleep_time > 0:
            print(f"\\nRate limit approaching, sleeping {sleep_time:.1f}s...")
            sleep(sleep_time)
            current_time = time.time()
            rate_limit_timestamps = []
    
    rate_limit_timestamps.append(current_time)
    
    try:
        # Generate instructions and Cypher with retry logic
        def _generate_instructions():
            return expander.expand_instructions(question)
        
        def _generate_cypher(inst):
            return generator.generate_cypher(inst)
        
        instructions = call_with_retry(_generate_instructions, max_attempts=5, base_delay=2.0)
        generated_cypher = call_with_retry(_generate_cypher, instructions, max_attempts=5, base_delay=2.0)
        
        # Count tokens for full prompt chain (reconstruct prompts for counting)
        from src.pipeline.prompts import CYPHER_PROMPT_TEMPLATE, INSTRUCTION_PROMPT_TEMPLATE, SCHEMA_SNIPPET
        instruction_prompt = INSTRUCTION_PROMPT_TEMPLATE.format(
            schema=SCHEMA_SNIPPET, question=question.strip()
        )
        cypher_prompt = CYPHER_PROMPT_TEMPLATE.format(
            schema=SCHEMA_SNIPPET, instructions=instructions.strip()
        )
        full_prompt = instruction_prompt + "\\n\\n" + cypher_prompt
        
        # Evaluate
        eval_result = evaluator.evaluate_single(
            question=question,
            gold_cypher=gold_cypher,
            generated_cypher=generated_cypher,
            prompt_text=full_prompt,
            model_name="gemini",
        )
        
        # Add metadata
        eval_result["id"] = record_id
        eval_result["question"] = question
        eval_result["gold_cypher"] = gold_cypher
        eval_result["instructions"] = instructions
        gemini_results.append(eval_result)
        
        # Checkpoint every 50 records
        if (i + 1) % 50 == 0:
            with checkpoint_file.open("w", encoding="utf-8") as f:
                for res in gemini_results:
                    f.write(json.dumps(res, ensure_ascii=False) + "\n")
            print(f"\nCheckpoint saved: {len(gemini_results)} records processed")
    
    except Exception as e:
        print(f"\\nError processing record {record_id}: {e}")
        gemini_results.append({
            "id": record_id,
            "question": question,
            "gold_cypher": gold_cypher,
            "syntactic_valid": False,
            "execution_success": False,
            "result_match": False,
            "generated_cypher": "",
            "error": f"Evaluation error: {type(e).__name__}: {e}",
            "latency_ms": 0,
            "input_tokens": 0,
            "output_tokens": 0,
            "gold_rows": [],
            "generated_rows": [],
        })

# Final checkpoint
with checkpoint_file.open("w", encoding="utf-8") as f:
    for res in gemini_results:
        f.write(json.dumps(res, ensure_ascii=False) + "\n")

print(f"\\nGemini evaluation complete: {len(gemini_results)} records")
gemini_metrics = evaluator.aggregate_metrics(gemini_results)
print("\\nGemini Metrics:")
for key, value in gemini_metrics.items():
    print(f"  {key}: {value:.2f}" if isinstance(value, float) else f"  {key}: {value}")


Evaluating Gemini: 100%|██████████| 10/10 [00:17<00:00,  1.71s/it]

\nGemini evaluation complete: 10 records
\nGemini Metrics:
  total: 10
  syntactic_validity_pct: 100.00
  execution_success_pct: 70.00
  semantic_accuracy_pct: 0.00
  avg_latency_ms: 867.72
  avg_input_tokens: 5164.10
  avg_output_tokens: 285.10
  total_input_tokens: 51641
  total_output_tokens: 2851


In [ ]:
# Qwen Base Model Evaluation

import torch
from unsloth import FastLanguageModel

# Create minimal Qwen prompt template
QWEN_MINIMAL_SCHEMA = """Graph schema:
- Nodes: Gene(symbol), Variant(name), Therapy(name), Disease(name), Biomarker
- Relationships: (Variant)-[:VARIANT_OF]->(Gene), (Therapy)-[:TARGETS]->(Gene), (Biomarker)-[:AFFECTS_RESPONSE_TO]->(Therapy)
- Properties: effect, disease_name, pmids, moa, ref_sources, ref_ids, ref_urls
- Return: Always include LIMIT, no parameters ($variables), use coalesce for arrays"""

QWEN_SYSTEM_PROMPT = f"""You are an expert Cypher query translator for oncology data.

{QWEN_MINIMAL_SCHEMA}

Rules:
- Return only Cypher query (no markdown, no explanation)
- Include RETURN clause and LIMIT
- Use toLower() for case-insensitive matching
- Wrap arrays with coalesce(..., []) before any()/all()
- For disease filters, use token-based CONTAINS matching"""

def format_qwen_prompt(question: str) -> str:
    """Format question using Qwen chat template."""
    return f"""<|im_start|>system
{QWEN_SYSTEM_PROMPT}
<|im_end|>
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

# Load Qwen model
print("Loading Qwen model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-4B-Instruct",
    max_seq_length=2048,
    dtype=torch.bfloat16,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)  # Enable inference optimizations
print("Qwen model loaded successfully")

# Create evaluator for Qwen (using Qwen's tokenizer for token counting)
class QwenEvaluator(Evaluator):
    """Evaluator that uses Qwen's tokenizer for token counting."""
    
    def __init__(self, validator, executor, qwen_tokenizer):
        self.validator = validator
        self.executor = executor
        self.qwen_tokenizer = qwen_tokenizer
    
    def evaluate_single(self, question, gold_cypher, generated_cypher, prompt_text, model_name):
        start_time = time.perf_counter()
        
        # Count tokens using Qwen's tokenizer
        # Note: Qwen tokenizer's encode method may or may not support add_special_tokens
        try:
            input_tokens = len(self.qwen_tokenizer.encode(prompt_text, add_special_tokens=False))
            output_tokens = len(self.qwen_tokenizer.encode(generated_cypher, add_special_tokens=False))
        except TypeError:
            # Fallback if add_special_tokens not supported
            input_tokens = len(self.qwen_tokenizer.encode(prompt_text))
            output_tokens = len(self.qwen_tokenizer.encode(generated_cypher))
        
        # Syntactic validation
        syntactic_valid, syntax_error = evaluate_cypher_syntax(generated_cypher, self.validator)
        
        # Execution
        execution_success = False
        execution_error = None
        generated_rows = []
        if syntactic_valid:
            execution_success, generated_rows, execution_error = evaluate_cypher_execution(
                generated_cypher, self.executor
            )
        
        # Get gold results
        _, gold_rows, _ = evaluate_cypher_execution(gold_cypher, self.executor)
        
        # Result comparison
        result_match = False
        if syntactic_valid and execution_success:
            result_match = compare_results(gold_rows, generated_rows)
        
        latency_ms = (time.perf_counter() - start_time) * 1000
        
        return {
            "syntactic_valid": syntactic_valid,
            "execution_success": execution_success,
            "result_match": result_match,
            "generated_cypher": generated_cypher,
            "error": syntax_error or execution_error,
            "latency_ms": latency_ms,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "gold_rows": gold_rows,
            "generated_rows": generated_rows,
        }
    
    def aggregate_metrics(self, results):
        return super().aggregate_metrics(results)

# Note: tokenizer will be defined after model loading
# qwen_evaluator will be created after model/tokenizer are loaded


In [ ]:
# Run Qwen Evaluation

qwen_results = []
checkpoint_file_qwen = EVAL_DIR / "qwen_results_checkpoint.jsonl"

# Load checkpoint if exists
if checkpoint_file_qwen.exists():
    print(f"Loading Qwen checkpoint from {checkpoint_file_qwen}")
    with checkpoint_file_qwen.open("r", encoding="utf-8") as f:
        checkpoint_records = [json.loads(line) for line in f]
        qwen_results = checkpoint_records
        processed_ids = {r["id"] for r in checkpoint_records}
        test_records_qwen = [r for r in test_records_qwen if r["id"] not in processed_ids]
        print(f"Resuming from checkpoint: {len(checkpoint_records)} already processed, {len(test_records_qwen)} remaining")

for i, record in enumerate(tqdm(test_records_qwen, desc="Evaluating Qwen")):
    question = record["question"]
    gold_cypher = record["cypher"]
    record_id = record["id"]
    
    try:
        # Format prompt
        prompt_text = format_qwen_prompt(question)
        
        # Generate Cypher
        inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract Cypher from response (after assistant tag)
        if "<|im_start|>assistant" in generated_text:
            generated_cypher = generated_text.split("<|im_start|>assistant")[-1].strip()
        else:
            # If no tag, assume the model output is the Cypher
            generated_cypher = generated_text.split("<|im_end|>")[0].strip()
        
        # Clean up markdown code fences if present
        if "```" in generated_cypher:
            lines = generated_cypher.split("\\n")
            # Remove code fence lines
            cleaned = [l for l in lines if not l.strip().startswith("```")]
            generated_cypher = "\\n".join(cleaned).strip()
        
        # Evaluate
        eval_result = qwen_evaluator.evaluate_single(
            question=question,
            gold_cypher=gold_cypher,
            generated_cypher=generated_cypher,
            prompt_text=prompt_text,
            model_name="qwen",
        )
        
        # Add metadata
        eval_result["id"] = record_id
        eval_result["question"] = question
        eval_result["gold_cypher"] = gold_cypher
        qwen_results.append(eval_result)
        
        # Checkpoint every 50 records
        if (i + 1) % 50 == 0:
            with checkpoint_file_qwen.open("w", encoding="utf-8") as f:
                for res in qwen_results:
                    f.write(json.dumps(res, ensure_ascii=False) + "\n")
            print(f"\nCheckpoint saved: {len(qwen_results)} records processed")
    
    except Exception as e:
        print(f"\\nError processing record {record_id}: {e}")
        qwen_results.append({
            "id": record_id,
            "question": question,
            "gold_cypher": gold_cypher,
            "syntactic_valid": False,
            "execution_success": False,
            "result_match": False,
            "generated_cypher": "",
            "error": f"Evaluation error: {type(e).__name__}: {e}",
            "latency_ms": 0,
            "input_tokens": 0,
            "output_tokens": 0,
            "gold_rows": [],
            "generated_rows": [],
        })

# Final checkpoint
with checkpoint_file_qwen.open("w", encoding="utf-8") as f:
    for res in qwen_results:
        f.write(json.dumps(res, ensure_ascii=False) + "\n")

print(f"\\nQwen evaluation complete: {len(qwen_results)} records")
qwen_metrics = qwen_evaluator.aggregate_metrics(qwen_results)
print("\\nQwen Metrics:")
for key, value in qwen_metrics.items():
    print(f"  {key}: {value:.2f}" if isinstance(value, float) else f"  {key}: {value}")


In [ ]:
# Results Analysis & Comparison

from datetime import datetime

# Aggregate metrics
gemini_metrics = evaluator.aggregate_metrics(gemini_results)
qwen_metrics = qwen_evaluator.aggregate_metrics(qwen_results)

# Create comparison table
comparison_data = {
    "Metric": [
        "Syntactic Validity (%)",
        "Execution Success (%)",
        "Semantic Accuracy (%)",
        "Avg Latency (ms)",
        "Avg Input Tokens",
        "Avg Output Tokens",
        "Total Input Tokens",
        "Total Output Tokens",
    ],
    "Gemini 2.5 Flash-Lite": [
        gemini_metrics.get("syntactic_validity_pct", 0),
        gemini_metrics.get("execution_success_pct", 0),
        gemini_metrics.get("semantic_accuracy_pct", 0),
        gemini_metrics.get("avg_latency_ms", 0),
        gemini_metrics.get("avg_input_tokens", 0),
        gemini_metrics.get("avg_output_tokens", 0),
        gemini_metrics.get("total_input_tokens", 0),
        gemini_metrics.get("total_output_tokens", 0),
    ],
    "Qwen3-4B Base": [
        qwen_metrics.get("syntactic_validity_pct", 0),
        qwen_metrics.get("execution_success_pct", 0),
        qwen_metrics.get("semantic_accuracy_pct", 0),
        qwen_metrics.get("avg_latency_ms", 0),
        qwen_metrics.get("avg_input_tokens", 0),
        qwen_metrics.get("avg_output_tokens", 0),
        qwen_metrics.get("total_input_tokens", 0),
        qwen_metrics.get("total_output_tokens", 0),
    ],
}

df_comparison = pd.DataFrame(comparison_data)
print("\\n" + "="*80)
print("BASELINE EVALUATION COMPARISON")
print("="*80)
print(df_comparison.to_string(index=False))
print("\\n")

# Calculate token efficiency improvement
gemini_total = gemini_metrics.get("total_input_tokens", 0) + gemini_metrics.get("total_output_tokens", 0)
qwen_total = qwen_metrics.get("total_input_tokens", 0) + qwen_metrics.get("total_output_tokens", 0)
if gemini_total > 0:
    token_reduction_pct = ((gemini_total - qwen_total) / gemini_total) * 100
    print(f"Token usage reduction (Qwen vs Gemini): {token_reduction_pct:.1f}%")
    print(f"  Gemini total: {gemini_total} tokens")
    print(f"  Qwen total: {qwen_total} tokens")


In [ ]:
# Identify Partial Matches (syntactic + execution pass but result mismatch)

def extract_partial_matches(results: list[dict[str, Any]], model_name: str) -> list[dict[str, Any]]:
    """Extract records where syntactic and execution pass but result doesn't match."""
    partial = []
    for r in results:
        if r["syntactic_valid"] and r["execution_success"] and not r["result_match"]:
            partial.append({
                "id": r["id"],
                "question": r["question"],
                "gold_cypher": r["gold_cypher"],
                "generated_cypher": r["generated_cypher"],
                "gold_rows": r.get("gold_rows", []),
                "generated_rows": r.get("generated_rows", []),
                "error": r.get("error"),
            })
    return partial

gemini_partial = extract_partial_matches(gemini_results, "gemini")
print("\\nPartial matches (syntax + exec pass, result mismatch):")
print(f"  Gemini: {len(gemini_partial)}")

# Only process Qwen if results are available
if 'qwen_results' in globals() and qwen_results:
    qwen_partial = extract_partial_matches(qwen_results, "qwen")
    print(f"  Qwen: {len(qwen_partial)}")
else:
    qwen_partial = []

# Save partial matches for inspection
partial_gemini_file = EVAL_DIR / "partial_matches_gemini.jsonl"
with partial_gemini_file.open("w", encoding="utf-8") as f:
    for record in gemini_partial:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print("\nPartial matches saved to:")
print(f"  {partial_gemini_file}")

if qwen_partial:
    partial_qwen_file = EVAL_DIR / "partial_matches_qwen.jsonl"
    with partial_qwen_file.open("w", encoding="utf-8") as f:
        for record in qwen_partial:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")
    print(f"  {partial_qwen_file}")


\nPartial matches (syntax + exec pass, result mismatch):
  Gemini: 7
\nPartial matches saved to:
  C:\Users\ishui\Desktop\OncoGraph\OncoGraph Agent\finetuning\eval\partial_matches_gemini.jsonl


In [24]:
# Detailed Result Inspection

def inspect_record(record: dict[str, Any], max_rows_display: int = 20):
    """Pretty-print a record for detailed inspection with Cypher queries paired with their results."""
    print("="*100)
    print(f"ID: {record.get('id', 'N/A')}")
    print(f"Question: {record.get('question', 'N/A')}")
    print("\\n" + "-"*100)
    
    # Compact status header
    status_parts = []
    status_parts.append("Syntax: " + ("✓" if record.get('syntactic_valid') else "✗"))
    status_parts.append("Exec: " + ("✓" if record.get('execution_success') else "✗"))
    status_parts.append("Match: " + ("✓" if record.get('result_match') else "✗"))
    status_parts.append(f"Latency: {record.get('latency_ms', 0):.0f}ms")
    status_parts.append(f"Tokens: {record.get('input_tokens', 0)}/{record.get('output_tokens', 0)}")
    print("STATUS: " + " | ".join(status_parts))
    if record.get('error'):
        print(f"ERROR: {record.get('error')}")
    
    gold_cypher = record.get('gold_cypher', '')
    gen_cypher = record.get('generated_cypher', '')
    gold_rows = record.get('gold_rows', [])
    gen_rows = record.get('generated_rows', [])
    
    # Show GOLD CYPHER with its results together
    print("\\n" + "="*100)
    print("📋 GOLD CYPHER (Expected Reference)")
    print("="*100)
    if gold_cypher:
        print("```cypher")
        for i, line in enumerate(gold_cypher.split('\\n'), 1):
            print(f"{line}")
        print("```")
    else:
        print("  (empty)")
    
    print(f"\\n📊 GOLD RESULTS: {len(gold_rows)} row(s)")
    print("-"*100)
    if gold_rows:
        for i, row in enumerate(gold_rows[:max_rows_display], 1):
            # Format row nicely as key: value pairs
            row_str = ", ".join([f"{k}={v!r}" for k, v in row.items()])
            print(f"  [{i:3d}] {row_str}")
        if len(gold_rows) > max_rows_display:
            print(f"  ... ({len(gold_rows) - max_rows_display} more rows)")
    else:
        print("  (no results)")
    
    # Show GENERATED CYPHER with its results together
    print("\\n" + "="*100)
    print("🔧 GENERATED CYPHER (Model Output)")
    print("="*100)
    if gen_cypher:
        print("```cypher")
        for i, line in enumerate(gen_cypher.split('\\n'), 1):
            print(f"{line}")
        print("```")
    else:
        print("  (empty)")
    
    print(f"\\n📊 GENERATED RESULTS: {len(gen_rows)} row(s)")
    print("-"*100)
    if gen_rows:
        for i, row in enumerate(gen_rows[:max_rows_display], 1):
            row_str = ", ".join([f"{k}={v!r}" for k, v in row.items()])
            print(f"  [{i:3d}] {row_str}")
        if len(gen_rows) > max_rows_display:
            print(f"  ... ({len(gen_rows) - max_rows_display} more rows)")
    else:
        print("  (no results)")
    
    # Show comparison only if both executed successfully but don't match
    if record.get('syntactic_valid') and record.get('execution_success') and not record.get('result_match'):
        print("\\n" + "="*100)
        print("🔍 RESULT COMPARISON")
        print("="*100)
        print(f"Row counts: Gold={len(gold_rows)}, Generated={len(gen_rows)}")
        
        if len(gold_rows) != len(gen_rows):
            print(f"⚠️  Row count mismatch!")
        
        # Identify differences
        if gold_rows and gen_rows:
            gold_set = {tuple(sorted(r.items())) for r in gold_rows}
            gen_set = {tuple(sorted(r.items())) for r in gen_rows}
            missing_in_gen = gold_set - gen_set
            extra_in_gen = gen_set - gold_set
            
            if missing_in_gen:
                print(f"\\n❌ Missing in generated ({len(missing_in_gen)} rows):")
                for j, row_tuple in enumerate(list(missing_in_gen)[:5], 1):
                    row_dict = dict(row_tuple)
                    row_str = ", ".join([f"{k}={v!r}" for k, v in row_dict.items()])
                    print(f"  {j}. {row_str}")
            
            if extra_in_gen:
                print(f"\\n➕ Extra in generated ({len(extra_in_gen)} rows):")
                for j, row_tuple in enumerate(list(extra_in_gen)[:5], 1):
                    row_dict = dict(row_tuple)
                    row_str = ", ".join([f"{k}={v!r}" for k, v in row_dict.items()])
                    print(f"  {j}. {row_str}")
            
            if not missing_in_gen and not extra_in_gen and len(gold_rows) == len(gen_rows):
                print("⚠️  Row sets are identical but comparison failed - possible harness issue")
        elif not gold_rows and not gen_rows:
            print("✓ Both queries returned empty results")
        else:
            print(f"⚠️  One query returned results, the other didn't")
    
    print("\\n")

# Create a summary DataFrame for easy browsing
def create_inspection_summary(results: list[dict[str, Any]], model_name: str) -> pd.DataFrame:
    """Create a readable summary DataFrame from results."""
    summary_data = []
    for r in results:
        # Safely handle error field (can be None, empty string, or actual error message)
        error_val = r.get("error") or ""
        error_str = error_val
        
        # Safely handle question field
        question_val = r.get("question") or ""
        question_str = question_val
        
        summary_data.append({
            "id": r.get("id", ""),
            "question": question_str,
            "syntactic_valid": "✓" if r.get("syntactic_valid") else "✗",
            "execution_success": "✓" if r.get("execution_success") else "✗",
            "result_match": "✓" if r.get("result_match") else "✗",
            "gold_rows": len(r.get("gold_rows", [])),
            "gen_rows": len(r.get("generated_rows", [])),
            "error": error_str,
        })
    return pd.DataFrame(summary_data)

print("\\n=== GEMINI RESULTS SUMMARY ===")
gemini_summary = create_inspection_summary(gemini_results, "gemini")
print(gemini_summary.to_string(index=False))

# Save readable summaries
gemini_summary.to_csv(EVAL_DIR / "gemini_results_summary.csv", index=False)

# Check if Qwen results are available
if 'qwen_results' in globals() and qwen_results:
    print("\\n\\n=== QWEN RESULTS SUMMARY ===")
    qwen_summary = create_inspection_summary(qwen_results, "qwen")
    print(qwen_summary.to_string(index=False))
    qwen_summary.to_csv(EVAL_DIR / "qwen_results_summary.csv", index=False)
    print(f"\\nReadable summaries saved to CSV files in {EVAL_DIR}")
else:
    print(f"\\n(Qwen results not available - skipping Qwen summary)")
    print(f"Gemini summary saved to: {EVAL_DIR / 'gemini_results_summary.csv'}")

# Display detailed view of mismatches
if 'gemini_partial' in globals() and gemini_partial:
    print("\\n" + "="*100)
    print("DETAILED INSPECTION: GEMINI PARTIAL MATCHES (syntax + exec pass, result mismatch)")
    print("="*100)
    for i, record in enumerate(gemini_partial[:3], 1):
        print(f"\\n--- Mismatch #{i} ---")
        inspect_record(record)

if 'qwen_partial' in globals() and qwen_partial:
    print("\\n\\n" + "="*100)
    print("DETAILED INSPECTION: QWEN PARTIAL MATCHES")
    print("="*100)
    for i, record in enumerate(qwen_partial[:3], 1):
        print(f"\\n--- Mismatch #{i} ---")
        inspect_record(record)


\n=== GEMINI RESULTS SUMMARY ===
               id                                                                                           question syntactic_valid execution_success result_match  gold_rows  gen_rows                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     error
F5.2-000167-S1-P2                           Show drugs with variant-based evidence in Chronic Myelogenous Leukaemia.               ✓                 ✓            ✗ 

In [ ]:
# Save Final Results

results_summary = {
    "timestamp": datetime.now().isoformat(),
    "test_set_size": len(test_records),
    "models": {
        "gemini_2.5_flash_lite": {
            "metrics": gemini_metrics,
            "full_results": gemini_results,
        },
    },
}

# Only include Qwen if results are available
if 'qwen_metrics' in globals() and 'qwen_results' in globals():
    results_summary["models"]["qwen3_4b_base"] = {
        "metrics": qwen_metrics,
        "full_results": qwen_results,
    }

results_file = EVAL_DIR / "evaluation_results.json"
with results_file.open("w", encoding="utf-8") as f:
    json.dump(results_summary, f, indent=2, ensure_ascii=False)

print(f"\\nFinal results saved to: {results_file}")
print("\\nEvaluation complete!")


In [ ]:
# Interactive Inspection Tool

def inspect_by_id(record_id: str, results: list[dict[str, Any]] | None = None):
    """Inspect a specific record by ID from either gemini_results or qwen_results."""
    if results is None:
        # Try both (only if they exist)
        all_results = gemini_results.copy() if 'gemini_results' in globals() else []
        if 'qwen_results' in globals():
            all_results.extend(qwen_results)
    else:
        all_results = results
    
    for record in all_results:
        if record.get("id") == record_id:
            inspect_record(record, max_rows_display=50)
            return
    print(f"Record with ID '{record_id}' not found.")

# Example usage:
# inspect_by_id("F5.2-000167-S1-P2")  # Uncomment and run to inspect a specific record

print("\\nUse inspect_by_id('RECORD_ID') to inspect a specific record in detail.")
print("Example: inspect_by_id('F5.2-000167-S1-P2')")
